In [1]:
import copy
#estado inicial
s_0=[[1,0,2],
     [6,3,4],
     [7,5,8]]
#estado final
s_f=[[0,1,2],
     [3,4,5],
     [6,7,8]]

In [2]:
#acciones
#las acciones se definieron a partir de a donde se puede mover el espacio y por que pieza
#En donde se representa de la siguiente manera (lugar donde se encuentra el espacio, accion que toma, pieza por la que se cambia)
#Ejemplo ([0,1],"down",3) 
def accion(estados):
    if 0 in estados[0]:
        if estados[0][0]==0:
            return [([0,0],"down",estados[1][0]),([0,0],"right",estados[0][1])]
        elif estados[0][1]==0:
            return[([0,1],"down",estados[1][1]),([0,1],"right",estados[0][2]),([0,1],"left",estados[0][0])]
        else:
            return[([0,2],"down",estados[1][2]),([0,2],"left",estados[0][1])]
    elif 0 in estados[1]:
        if estados[1][0]==0:
            return[([1,0],"up",estados[0][0]),([1,0],"down",estados[2][0]),([1,0],"right",estados[1][1])]
        elif estados[1][1]==0:
            return[([1,1],"up",estados[0][1]),([1,1],"down",estados[2][1]),([1,1],"right",estados[1][2]),([1,1],"left",estados[1][0])]
        else: 
            return[([1,2],"up",estados[0][2]),([1,2],"down",estados[2][2]),([1,2],"left",estados[1][1])]
    else:
        if estados[2][0]==0:
            return [([2,0],"up",estados[1][0]),([2,0],"right",estados[2][1])]
        elif estados[2][1]==0:
            return[([2,1],"up",estados[1][1]),([2,1],"right",estados[2][2]),([2,1],"left",estados[2][0])]
        else:
            return[([2,2],"up",estados[1][2]),([2,2],"left",estados[2][1])]

accion(s_0)        

[([0, 1], 'down', 3), ([0, 1], 'right', 2), ([0, 1], 'left', 1)]

In [3]:
#modelo de transicion
def movimiento(estado,accion):
    #conocer la pieza que vamos a mover 
    pieza_mover=accion[2]
    #saber donde esta la pieza por la que  vamos a mover el estado
    if accion[1]=="down":
        lugar_pieza=[accion[0][0]+1,accion[0][1]]
    elif accion[1]=="up":
        lugar_pieza=[accion[0][0]-1,accion[0][1]]
    elif accion[1]=="right":
        lugar_pieza=[accion[0][0],accion[0][1]+1]
    else:
        lugar_pieza=[accion[0][0],accion[0][1]-1]
        #crear nuevo estado
    nuevo_estado=copy.deepcopy(estado)   
    nuevo_estado[lugar_pieza[0]][lugar_pieza[1]]=0 #remplazo por el espacio
    nuevo_estado[accion[0][0]][accion[0][1]]=pieza_mover#remplazo por la pieza
    return nuevo_estado 
movimiento(s_0,accion(s_0)[0])

[[1, 3, 2], [6, 0, 4], [7, 5, 8]]

In [4]:
#clase nodos
#Esta clase nos ayudara a saber el estado, los padres, la acción y el costo
class Node():
    def __init__(self):
        self.state =None
        self.parent = None
        self.action = None
        self.cost = 0

In [5]:
#expancion de nodos
def expande(nodo):
    estado=nodo.state
    for a in accion(estado):
        #conocer el nuevo estado
        nuevo_estado=movimiento(estado,a)
        costo=nodo.cost+1 #aumentar el costo por la accion tomada
        # crear el nuevo nodo
        new_nodo=Node()
        new_nodo.state=nuevo_estado
        new_nodo.parent=nodo
        new_nodo.action=a[1]
        new_nodo.cost=costo
        yield new_nodo

#pruebas        
#prueba_nodo=Node()
#prueba_nodo.state=s_0

#list(expande(prueba_nodo))[2].cost

In [6]:
#Busqueda
#En esta definicion de busqueda todo depende de como definamos nuestra frontera
estado_inicial=Node()
estado_inicial.state=s_0
nodos=[]#nodos tocados
frontera=[estado_inicial]
reach={str(estado_inicial):estado_inicial}#nodos alcanzados
def busqueda(estado):
    while len(frontera)>0:
        x=sorted(frontera,key=lambda node: node.cost)#ordena la frontera de menor a mayor
        nodo=x.pop(0)#saca el primer elemento 
        
        nodos.append(nodo)#agregamos el nodo a la lista nodos
        #eliminamos el elemento que sacamos de x de la frontera
        for i in frontera:
            if nodo==i:
                frontera.remove(nodo)
        #revisa si ya llegamos a la meta        
        if nodo.state==s_f:
            #break
            return nodos[-1].state
        #expandimos el nodo y revisamos si no esta en reach(alcanzados) para asi no caer en un ciclo y agregarlos a la frontera
        for hijos in expande(nodo):
            estado_hijo=hijos.state
            if str(estado_hijo) not in reach.keys():   
                reach[str(estado_hijo)]=hijos
                frontera.append(hijos)   
busqueda(estado_inicial)    

[[0, 1, 2], [3, 4, 5], [6, 7, 8]]

In [7]:
#Esta parte del programa nos regresara el camino que se tomo para llegar a la solucion
padre=nodos[-1].parent#saber cual es el nodo padre de la solucion
nodos_tomados=[nodos[-1],padre]#aqui se guardara los nodos tomados
#nodo_padre=[]
def camino(nodos):
    while estado_inicial not in nodos_tomados:
        nodo_padre=nodos_tomados[-1].parent#aqui obtenemos el padre de cada nodo
        nodos_tomados.append(nodo_padre)
camino(nodos)        

In [49]:
camino=sorted(nodos_tomados,key=lambda node: node.cost)#organizamos el camino de acuerdo a los costos para que se impriman en orden
k=0
#Ahora vamos a imprimir de forma ordenada el camino que toma para llegar a la solución
for i in camino:
    j=0
    print(k)
    while j<3:
        print( i.state[j])
        j+=1
    k+=1    

0
[1, 0, 2]
[6, 3, 4]
[7, 5, 8]
1
[1, 3, 2]
[6, 0, 4]
[7, 5, 8]
2
[1, 3, 2]
[6, 5, 4]
[7, 0, 8]
3
[1, 3, 2]
[6, 5, 4]
[7, 8, 0]
4
[1, 3, 2]
[6, 5, 0]
[7, 8, 4]
5
[1, 3, 2]
[6, 0, 5]
[7, 8, 4]
6
[1, 0, 2]
[6, 3, 5]
[7, 8, 4]
7
[1, 2, 0]
[6, 3, 5]
[7, 8, 4]
8
[1, 2, 5]
[6, 3, 0]
[7, 8, 4]
9
[1, 2, 5]
[6, 3, 4]
[7, 8, 0]
10
[1, 2, 5]
[6, 3, 4]
[7, 0, 8]
11
[1, 2, 5]
[6, 3, 4]
[0, 7, 8]
12
[1, 2, 5]
[0, 3, 4]
[6, 7, 8]
13
[1, 2, 5]
[3, 0, 4]
[6, 7, 8]
14
[1, 2, 5]
[3, 4, 0]
[6, 7, 8]
15
[1, 2, 0]
[3, 4, 5]
[6, 7, 8]
16
[1, 0, 2]
[3, 4, 5]
[6, 7, 8]
17
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]


In [8]:
len(nodos)

13000